In [ ]:
include("./prepare_data.jl")
include("./model.jl")

import .Model
import Flux

In [ ]:
using CUDA 
device = Flux.gpu

flux_model = Model.model |> device

In [ ]:
function flux_loss(model, x, y)
    ŷ = model(x)
    Flux.crossentropy(ŷ, y)
end

In [ ]:
using ProgressMeter
using MLUtils 
# include("./config.jl")
# import .ModelCfgs

data_loader = MLUtils.DataLoader(DataSet(ModelCfgs.D), batchsize=32) |> device
 opt_state  = Flux.setup(Flux.Adam(0.01),flux_model)


In [ ]:
losses = []
epochs = 500
@showprogress for epoch in 1:epochs 
                    for (x,y) in data_loader
                        l = flux_loss(flux_model, x, y)
                        push!(losses, l)
                        Flux.train!(flux_loss, flux_model, [(x, y)], opt_state)
                    end
end

In [ ]:
using Dates
now = Dates.now()
date_now = Dates.value(now)


In [ ]:
cpu_model = Flux.cpu(flux_model)

using BSON: @save
@save string("./output/mymodel_",date_now,".bson") cpu_model

In [ ]:
using Flux, BSON
BSON.@load string("./output/mymodel_",date_now,".bson") cpu_model

In [ ]:
Flux.params(cpu_model )